<a href="https://colab.research.google.com/github/rashmi05pathak/CS6910_Assignment_2/blob/main/CS6910_Assignment2B_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install wandb

     |████████████████████████████████| 1.8 MB 5.3 MB/s 
     |████████████████████████████████| 181 kB 47.3 MB/s 
     |████████████████████████████████| 144 kB 41.5 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=46eab0ec39aa67821528ee273bd8d799b705acf9f656ec4f19d161096050bf9a
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [3]:
import numpy as np
import pandas as pd
import os
import keras
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import cv2
import wandb
from wandb.keras import WandbCallback

In [4]:
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.xception import Xception

In [5]:
from keras.applications.resnet import ResNet50

In [6]:
%%capture
!curl -SL https://storage.googleapis.com/wandb_datasets/nature_12K.zip > nature_12K.zip
!unzip nature_12K.zip

train_dir='inaturalist_12K/train/'
test_dir='inaturalist_12K/val/'
categories=['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']

In [7]:
def customized_dataGenerator(h,w):
    img_height,img_width= h ,w
    #data generators for train and validation set
    
    augment= ImageDataGenerator(
             rotation_range=30,
             width_shift_range=0.2,
             height_shift_range=0.2,
             shear_range=0.2,
             zoom_range=0.2,
             horizontal_flip=True,
             rescale=1.0 / 255,
             validation_split=0.1
             )

    datagen = ImageDataGenerator(
              rescale=1.0 / 255,
              validation_split=0.1
              )

    train_set = datagen.flow_from_directory(
                train_dir,
                target_size=(img_height, img_width),
                batch_size=32,
                color_mode='rgb',
                class_mode='sparse',
                shuffle=True,
                subset='training',
                seed=95,
                )
    aug_set= augment.flow_from_directory(
            train_dir,
            target_size=(img_height, img_width),
            batch_size=32,
            color_mode='rgb',
            class_mode='sparse',
            shuffle=True,
            subset='training',
            seed=95,
           )

    val_set= datagen.flow_from_directory(
              train_dir,
              target_size=(img_height, img_width),
              color_mode='rgb',
              class_mode='sparse',
              shuffle=True,
              subset='validation',
              seed=95,
              )
    return train_set, aug_set, val_set


In [8]:
sweep_config = {
  'name': 'sweep2B5b',  
  'method': 'grid',
  'parameters': {
        
        
        'fc_size':{
            'values':[32,64,128]
        },
        'batchnorm':{
            'values':['yes','no']
        },
        'augmentation':{
            'values':['yes','no']   
        },
        'droprate':{
            'values':[0.3,0.4,0.5]
        },
        'pre_train':{
            'values':['inceptionv3','inceptionresnetv2','Xception','resnet50']
        }

           
    }

}

#this config gives best val accuracy
config_defaults={
    'fc_size':32,
    'batchnorm':'yes',
    'augmentation':'yes',
    'droprate':0.3,
    'pre_train':'inceptionv3'
}

In [10]:
#config_defaults

In [11]:
def PreTrain():

  wandb.init(config=config_defaults)
  configs=wandb.config

  configs=config_defaults

  batchnorm=configs['batchnorm']
  droprate=configs['droprate']
  augmentation=configs['augmentation']
  fc_size=configs['fc_size']
  pre_train = configs['pre_train']

  wandb.run.name='model_'+pre_train+'_fc_2_size_'+str(fc_size)+'_droprate_'+str(droprate)+ str('_bn_' if batchnorm=='yes' else '') +str('augment' if augmentation=='yes' else '')


  if pre_train == 'inceptionv3':
    img_height = 300
    img_width = 300
    base_model = InceptionV3(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

  if pre_train == 'inceptionresnetv2':
    img_height = 300
    img_width = 300
    base_model = InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

  if pre_train == 'resnet50':
    img_height = 224
    img_width = 224
    base_model = ResNet50(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

  if pre_train == 'Xception':
    img_height = 300
    img_width = 300
    base_model = Xception(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))  


  for layers in base_model.layers:
    layers.trainable = False

  model = keras.Sequential([
      tf.keras.Input(shape=(img_height, img_width,3,)),
      base_model,
      Flatten(),
      Dense(fc_size,activation='relu'),
      
  ])
  if batchnorm == 'yes':
    model.add(BatchNormalization())
  model.add(Dropout(droprate))
  model.add(Dense(fc_size, activation='relu'))
  model.add(Dropout(droprate))
  model.add(Dense(10 ,activation='softmax'))
  
  model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=[tf.keras.losses.SparseCategoricalCrossentropy()],
    metrics=['accuracy'],
  )
  
  train_set, aug_set, val_set = customized_dataGenerator(img_height,img_width)

  if augmentation =='no' :
    #hist=model.fit(train_set,epochs=10,validation_data=val_set,callbacks=[WandbCallback()]) 
    hist=model.fit(train_set,epochs=10,validation_data=val_set)
  
  else:
    #hist=model.fit(aug_set,epochs=10,validation_data=val_set,callbacks=[WandbCallback()])
    hist=model.fit(aug_set,epochs=10,validation_data=val_set)


  val_acc=max(hist.history['val_accuracy'])
  params={'batch_norm':batchnorm,'augmentation':augmentation,'dropout':droprate,'pre_trained_model':pre_train,'val_acc':val_acc}
  #wandb.log(params)
  

In [ ]:
#Ignore this and proceed
#sweep_id=wandb.sweep(sweep_config,entity='rashmipathak',project='assignment_2B')
#wandb.agent(sweep_id, PreTrain)   

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: zrxw5ela
Sweep URL: https://wandb.ai/rashmipathak/assignment_2B/sweeps/zrxw5ela


wandb: Agent Starting Run: vl179ozp with config:
wandb: 	augmentation: yes
wandb: 	batchnorm: yes
wandb: 	droprate: 0.3
wandb: 	fc_size: 32
wandb: 	pre_train: inceptionv3
wandb: Currently logged in as: rashmipathak (use `wandb login --relogin` to force relogin)


219070464/219055592 [==============================] - 1s 0us/step
Found 9000 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/10
282/282 [==============================] - 324s 1s/step - loss: 1.3154 - accuracy: 0.5811 - val_loss: 0.6184 - val_accuracy: 0.8068
Epoch 2/10
282/282 [==============================] - 294s 1s/step - loss: 0.8805 - accuracy: 0.7206 - val_loss: 0.6271 - val_accuracy: 0.8128
Epoch 3/10
282/282 [==============================] - 287s 1s/step - loss: 0.8192 - accuracy: 0.7489 - val_loss: 0.5622 - val_accuracy: 0.8208
Epoch 4/10
282/282 [==============================] - 286s 1s/step - loss: 0.7636 - accuracy: 0.7594 - val_loss: 0.5623 - val_accuracy: 0.8158
Epoch 5/10
282/282 [==============================] - 286s 1s/step - loss: 0.7407 - accuracy: 0.7680 - val_loss: 0.5364 - val_accuracy: 0.8318
Epoch 6/10
282/282 [==============================] - 286s 1s/step - loss: 0.7240 - accura

...

Found 9000 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/10
282/282 [==============================] - 298s 1s/step - loss: 1.3044 - accuracy: 0.5893 - val_loss: 0.6436 - val_accuracy: 0.8058
Epoch 2/10
282/282 [==============================] - 286s 1s/step - loss: 0.8885 - accuracy: 0.7234 - val_loss: 0.5660 - val_accuracy: 0.8158
Epoch 3/10
282/282 [==============================] - 286s 1s/step - loss: 0.8042 - accuracy: 0.7468 - val_loss: 0.5863 - val_accuracy: 0.8238
Epoch 4/10
282/282 [==============================] - 286s 1s/step - loss: 0.7695 - accuracy: 0.7599 - val_loss: 0.5685 - val_accuracy: 0.8268
Epoch 5/10
282/282 [==============================] - 286s 1s/step - loss: 0.7437 - accuracy: 0.7634 - val_loss: 0.5518 - val_accuracy: 0.8228
Epoch 6/10
282/282 [==============================] - 286s 1s/step - loss: 0.7105 - accuracy: 0.7683 - val_loss: 0.5088 - val_accuracy: 0.8328
Epoch 7/10
282

wandb: Agent Starting Run: jrpi2wo8 with config:
wandb: 	augmentation: yes
wandb: 	batchnorm: yes
wandb: 	droprate: 0.3
wandb: 	fc_size: 32
wandb: 	pre_train: resnet50


Found 9000 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/10
282/282 [==============================] - 298s 1s/step - loss: 1.3021 - accuracy: 0.5907 - val_loss: 0.6143 - val_accuracy: 0.8148
Epoch 2/10
282/282 [==============================] - 285s 1s/step - loss: 0.8907 - accuracy: 0.7223 - val_loss: 0.5709 - val_accuracy: 0.8178
Epoch 3/10
282/282 [==============================] - 284s 1s/step - loss: 0.8083 - accuracy: 0.7477 - val_loss: 0.5919 - val_accuracy: 0.8078
Epoch 4/10
282/282 [==============================] - 284s 1s/step - loss: 0.7745 - accuracy: 0.7572 - val_loss: 0.5510 - val_accuracy: 0.8168
Epoch 5/10
282/282 [==============================] - 285s 1s/step - loss: 0.7370 - accuracy: 0.7664 - val_loss: 0.5266 - val_accuracy: 0.8338
Epoch 6/10
282/282 [==============================] - 285s 1s/step - loss: 0.7042 - accuracy: 0.7748 - val_loss: 0.5150 - val_accuracy: 0.8368
Epoch 7/10
282

wandb: Agent Starting Run: fthrelam with config:
wandb: 	augmentation: yes
wandb: 	batchnorm: yes
wandb: 	droprate: 0.3
wandb: 	fc_size: 64
wandb: 	pre_train: inceptionv3


Found 9000 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/10
282/282 [==============================] - 298s 1s/step - loss: 1.3046 - accuracy: 0.5883 - val_loss: 0.6647 - val_accuracy: 0.7928
Epoch 2/10
282/282 [==============================] - 287s 1s/step - loss: 0.8731 - accuracy: 0.7251 - val_loss: 0.5731 - val_accuracy: 0.8218
Epoch 3/10
282/282 [==============================] - 288s 1s/step - loss: 0.8059 - accuracy: 0.7508 - val_loss: 0.5580 - val_accuracy: 0.8288
Epoch 4/10
282/282 [==============================] - 289s 1s/step - loss: 0.7596 - accuracy: 0.7600 - val_loss: 0.5326 - val_accuracy: 0.8308
Epoch 5/10
282/282 [==============================] - 289s 1s/step - loss: 0.7245 - accuracy: 0.7663 - val_loss: 0.5079 - val_accuracy: 0.8418
Epoch 6/10
282/282 [==============================] - 288s 1s/step - loss: 0.7082 - accuracy: 0.7747 - val_loss: 0.5053 - val_accuracy: 0.8478
Epoch 7/10
232

In [14]:
# Defining model with best configs to evaluate on test set
configs =config_defaults
batchnorm=configs['batchnorm']
droprate=configs['droprate']
augmentation=configs['augmentation']
fc_size=configs['fc_size']
pre_train = configs['pre_train']


if pre_train == 'inceptionv3':
    img_height = 300
    img_width = 300
    base_model = InceptionV3(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

if pre_train == 'inceptionresnetv2':
    img_height = 300
    img_width = 300
    base_model = InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

if pre_train == 'resnet50':
    img_height = 224
    img_width = 224
    base_model = ResNet50(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))

if pre_train == 'Xception':
    img_height = 300
    img_width = 300
    base_model = Xception(include_top=False, weights='imagenet',input_shape=(img_height, img_width,3))  


for layers in base_model.layers:
    layers.trainable = False

model = keras.Sequential([
      tf.keras.Input(shape=(img_height, img_width,3,)),
      base_model,
      Flatten(),
      Dense(fc_size,activation='relu'),
      
  ])
if batchnorm == 'yes':
    model.add(BatchNormalization())
model.add(Dropout(droprate))
model.add(Dense(fc_size, activation='relu'))
model.add(Dropout(droprate))
model.add(Dense(10 ,activation='softmax'))
  
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=[tf.keras.losses.SparseCategoricalCrossentropy()],
    metrics=['accuracy'],
  )
#Getting training data and validation data using keras library and applying augmentation there
train_set, aug_set, val_set = customized_dataGenerator(img_height,img_width)
if augmentation =='no' :
    #hist=model.fit(train_set,epochs=10,validation_data=val_set,callbacks=[WandbCallback()]) 
    hist=model.fit(train_set,epochs=10,validation_data=val_set)
  
else:
    #hist=model.fit(aug_set,epochs=10,validation_data=val_set,callbacks=[WandbCallback()])
    hist=model.fit(aug_set,epochs=10,validation_data=val_set)


87924736/87910968 [==============================] - 1s 0us/step
Found 9000 images belonging to 10 classes.
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/10
282/282 [==============================] - 351s 1s/step - loss: 1.5455 - accuracy: 0.4913 - val_loss: 0.7212 - val_accuracy: 0.7768
Epoch 2/10
282/282 [==============================] - 312s 1s/step - loss: 1.0984 - accuracy: 0.6542 - val_loss: 0.6946 - val_accuracy: 0.7928
Epoch 3/10
282/282 [==============================] - 310s 1s/step - loss: 0.9954 - accuracy: 0.6892 - val_loss: 1.1938 - val_accuracy: 0.7237
Epoch 4/10
282/282 [==============================] - 311s 1s/step - loss: 0.9318 - accuracy: 0.7092 - val_loss: 0.5972 - val_accuracy: 0.8068
Epoch 5/10
282/282 [==============================] - 315s 1s/step - loss: 0.9042 - accuracy: 0.7194 - val_loss: 0.6356 - val_accuracy: 0.8018
Epoch 6/10
282/282 [==============================] - 313s 1s/step - loss: 0.8872 - accuracy

In [15]:
#data generator for test set

test_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
  )

test_set = test_datagen.flow_from_directory(
      test_dir,
      target_size=(img_height, img_width),
      color_mode='rgb',
      class_mode='sparse',
      shuffle=True,
      seed=123,
  )

#evaluating model on test set
model.evaluate(test_set)

Found 2000 images belonging to 10 classes.
63/63 [==============================] - 35s 545ms/step - loss: 0.5444 - accuracy: 0.8295


[0.5444099307060242, 0.8295000195503235]